In [ ]:
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import librosa
from IPython.display import Audio
from scipy.io.wavfile import read

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import confusion_matrix, classification_report

import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import LSTM, Dense, Dropout, Input
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.optimizers import Adam

In [ ]:
tess_dir = Path("TESS_Data")
tess_files = list(tess_dir.glob('**/*.wav'))
tess_labels = [path.parent.name for path in tess_files]
audio_df = pd.DataFrame({'audio_file': tess_files, 'emotion': tess_labels}).sample(frac=1, random_state= 42).reset_index(drop=True)

In [ ]:
print("Total files:", len(audio_df))
print(audio_df['emotion'].value_counts())
print("Emotions:", ", ".join(audio_df['emotion'][:5].tolist()))

***Audio Processing Functions with Some Additional Tunings***

In [ ]:
def load_audio(path):
    try:
        data, sample_rate = librosa.load(path, duration=2.5, offset=0.6)
        data = librosa.util.normalize(data)
        return data, sample_rate
    except Exception as e:
        print(f"Error loading file {path}: {e}")
        return None, None

In [ ]:
def add_noise(data):
    noise_value = 0.015 * np.random.uniform() * np.amax(data)
    return data + noise_value * np.random.normal(size=data.shape[0])

In [ ]:
def stretch_audio(data, rate=0.9):
    return librosa.effects.time_stretch(data, rate=rate)

In [ ]:
def change_pitch(data, sampling_rate, pitch_factor=3):
    return librosa.effects.pitch_shift(data, sr=sampling_rate, n_steps=pitch_factor)

***Feature Extraction***

In [ ]:
# Feature Extraction as MFCC sequences
def extract_features(data, sample_rate, n_mfcc=40, max_pad_len=100):
    # Extract MFCC (as sequence)
    mfcc = librosa.feature.mfcc(y=data, sr=sample_rate, n_mfcc=n_mfcc)
    mfcc = mfcc.T  # Shape: (timesteps, features)
    if mfcc.shape[0] < max_pad_len:
        pad_width = max_pad_len - mfcc.shape[0]
        mfcc = np.pad(mfcc, ((0,pad_width),(0,0)), mode='constant')
    else:
        mfcc = mfcc[:max_pad_len, :]
    return mfcc

In [ ]:
def export_features(path):
    data, sample_rate = load_audio(path)
    if data is None:
        return np.array([])
    result = [extract_features(data, sample_rate)]
    noisy_data = add_noise(data)
    result.append(extract_features(noisy_data, sample_rate))
    stretched_pitch = change_pitch(stretch_audio(data), sample_rate)
    result.append(extract_features(stretched_pitch, sample_rate))
    return np.array(result)

In [ ]:
X_data, y_data = [], []
for path, emotion in zip(audio_df.audio_file, audio_df.emotion):
    features = export_features(path)
    if features.size > 0:
        for seq in features:
            X_data.append(seq)
            y_data.append(emotion)

In [ ]:
X_data = np.array(X_data)
y_data = np.array(y_data).reshape(-1, 1)

print("Feature shape:", X_data.shape)
print("Labels shape:", y_data.shape)

***Checking Some Audio Samples With Above Tunings & Plotting Them***

In [ ]:
rate, speech = read(audio_df['audio_file'][2342])
print(audio_df['emotion'][2342])
Audio(speech, rate=rate, autoplay=False)

In [ ]:
rate, speech = read(audio_df['audio_file'][20])
print(audio_df['emotion'][20])
Audio(speech, rate=rate, autoplay=False)

In [ ]:
figure = plt.figure(figsize=(13,6))
audio_speech,rate = librosa.load(audio_df['audio_file'][120])
print(audio_df['emotion'][120])
librosa.display.waveshow(audio_speech, sr=rate, color = 'orange')
Audio(audio_speech, rate=rate)

In [ ]:
figure = plt.figure(figsize=(13,6))
audio_speech,rate = librosa.load(audio_df['audio_file'][10])
print(audio_df['emotion'][10])
librosa.display.waveshow(audio_speech, sr=rate, color = 'green')
Audio(audio_speech, rate=rate)

In [ ]:
figure = plt.figure(figsize=(13,6))
plt.title("Tune 1: Voice With Some Noise")
audio_speech,sample_rate = librosa.load(audio_df['audio_file'][2000])
print(audio_df['emotion'][2000])
noise_injection = add_noise(audio_speech)
librosa.display.waveshow(noise_injection, sr=sample_rate)
Audio(noise_injection, rate=sample_rate)

In [ ]:
figure = plt.figure(figsize=(13,6))
plt.title("Tune 2: Streched Voice")
audio_speech,sample_rate = librosa.load(audio_df['audio_file'][2000])
print(audio_df['emotion'][2000])
stretching_audio = stretch_audio(audio_speech)
librosa.display.waveshow(stretching_audio, sr=sample_rate, color='red')
Audio(stretching_audio, rate=sample_rate)

***Spectrogram of a Audio Sample***

In [ ]:
figure = plt.figure(figsize=(13,6))
audio_speech, rate = librosa.load(audio_df['audio_file'][2000])
stft_audio = librosa.stft(audio_speech)
Db_audio = librosa.amplitude_to_db(abs(stft_audio))
librosa.display.specshow(Db_audio, sr=rate, x_axis='time', y_axis='hz')
plt.title('Spectrogram')
plt.colorbar()
plt.show()

***Visualization of MFCCs of a Audio Sample***

In [ ]:
audio_file, sr = librosa.load(audio_df['audio_file'][2000])
mfccs = librosa.feature.mfcc(y=audio_file, sr=sr, n_mfcc=13)

In [ ]:
plt.figure(figsize=(13, 6))
librosa.display.specshow(mfccs, sr=sr, x_axis='time', cmap='cool')
plt.title('Mel-Frequency Cepstral Coefficients (MFCCs)')
plt.colorbar(format='%+2.0f dB')
plt.xlabel('Time (s)')
plt.ylabel('MFCC Coefficients')
plt.show()

***Preprocessing***

In [ ]:
encoder_label = OneHotEncoder(sparse_output=False)

Y = encoder_label.fit_transform(y_data)
print(Y.shape)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_data, Y, test_size=0.3, random_state=42, shuffle=True)

In [ ]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

In [ ]:
num_timesteps = X_train.shape[1]
num_features = X_train.shape[2]

scaler = StandardScaler()
X_train_flat = X_train.reshape(-1, num_features)
X_test_flat = X_test.reshape(-1, num_features)

In [ ]:
print(X_train_flat.shape)
print(X_test_flat.shape)

***Model Creation (LSTM)***

In [ ]:
# Reshaping for LSTM
X_train_scaled = scaler.fit_transform(X_train_flat).reshape(-1, num_timesteps, num_features)
X_test_scaled = scaler.transform(X_test_flat).reshape(-1, num_timesteps, num_features)

In [ ]:
print(X_train_scaled.shape)
print(X_test_scaled.shape)

In [ ]:
def create_lstm_model(input_shape, num_classes):
    model = Sequential([
        Input(shape=input_shape),
        LSTM(128, return_sequences=True),
        Dropout(0.2),
        LSTM(64),
        Dropout(0.3),
        Dense(num_classes, activation='softmax')
    ])
    optimizer = Adam(learning_rate=0.001)
    model.compile(optimizer= optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [ ]:
lstm_model = create_lstm_model((num_timesteps, num_features), Y.shape[1])
lstm_model.summary()

In [ ]:
callbacks = [
    ModelCheckpoint('speech-emotion-recognition.keras', monitor='val_loss', save_best_only=True, verbose=1),
    EarlyStopping(monitor='val_loss', patience=6, restore_best_weights=True, verbose=1)
]

history = lstm_model.fit(
    X_train,
    y_train,
    batch_size=64,
    epochs=50,
    validation_data=(X_test, y_test),
    callbacks=callbacks
)


***Visualization of Accuracy***

In [ ]:
plt.figure(figsize=(15, 5))
plt.plot(history.history['accuracy'], label='Train Accuracy', color='red', marker='o', linestyle='-')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy', color='blue', marker='o', linestyle='-')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

In [ ]:
plt.figure(figsize=(15, 5))
plt.plot(history.history['loss'], label='Train Loss', color='red', marker='o', linestyle='-')
plt.plot(history.history['val_loss'], label='Validation Loss', color='blue', marker='o', linestyle='-')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

***Evaluation***

In [ ]:
model = load_model('speech-emotion-recognition.keras')
results = model.evaluate(X_test_scaled, y_test)
print('Test loss:', results[0])
print('Test accuracy:', results[1])

In [ ]:
test_prediction = model.predict(X_test_scaled)
y_pred = encoder_label.inverse_transform(test_prediction)
y_test_inv = encoder_label.inverse_transform(y_test)

In [ ]:
print("Predicted labels:", y_pred[:10])
print("Actual labels   :", y_test_inv[:10])

***Confusion Matrix***

In [ ]:
conf_matrix = confusion_matrix(np.argmax(y_test, axis=1), np.argmax(test_prediction, axis=1))
plt.figure(figsize=(13, 6))
sns.heatmap(conf_matrix, linecolor='white', cmap='Blues', annot=True, fmt='d')
plt.title('Confusion Matrix')
plt.xlabel('Predicted Labels')
plt.ylabel('Actual Labels')
plt.show()

***Classification Report***

In [ ]:
print(classification_report(y_test_inv, y_pred))